# Calculate elastic constant tensors on Matlantis
**Instruction**  
The aim of this notebook to calculate elastic constant tensors on Matlantis.  
PreFerred Potential (PFP) is commercially available on a cloud software Matlantis.  
CHGnet is installed using `pip install chgnet`.  
Set `atoms.calc = Calculator (PFP or CHGNet)` for potential energy calculation.  
  
**Note**  
Please assign raw cif folder, optimized cif folder, and csv filename. 

In [ ]:
import logging, pathlib, glob, os, csv, utility, ase
import numpy as np
import pandas as pd
from ase.io import read, write
from ase.constraints import ExpCellFilter
from ase.optimize import BFGS
import warnings
warnings.simplefilter('ignore')
from chgnet.model.dynamics import CHGNetCalculator
os.environ['PFP_INFERENCE_METHOD_TYPE'] = 'PFVM'
from matlantis_features.features.common import BFGSASEOptFeature
from matlantis_features.features.elasticity.elastic_tensor import ElasticTensorFeature
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

print(matlantis_features.__version__)
print(pfp_api_client.__version__)

In [ ]:
files = glob.glob('./*.cif') ## Assign folder
len(files)

In [ ]:
########### Assign csv filename #############
path_results = 'FILE_PATH.csv"
path_error = 'FILE_PATH.csv"
#############################################

columns = ['refcode', 
           'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
           'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 
           'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 
           'exp_a', 'exp_b', 'exp_c', 'exp_alpha', 'exp_beta', 'exp_gamma', 'exp_V', 
           'opt_a', 'opt_b', 'opt_c', 'opt_alpha', 'opt_beta', 'opt_gamma', 'opt_V']

df = pd.DataFrame(columns=columns)
df_error = pd.DataFrame(columns=["refcode"])

# Prepare dataframe (Warning!!! Please do this just the first time)
df.to_csv(path_results, index=False)
df_error.to_csv(path_error, index=False)

skip_list = []

In [ ]:
%%time

# Automated calclation of Young's modulus 
for i, file in enumerate(files[:]):
    refcode = os.path.basename(file).rstrip('.cif')
    df = pd.read_csv(path_results)
    df_error = pd.read_csv(path_error)
    if (refcode not in list(df['refcode']) and 
        refcode not in list(df_error['refcode']) and 
        refcode not in skip_list):
        try:
            print(i, refcode)
            atoms_in = read(file)
            atoms = atoms_in.copy()
            
            # ----- PFP model -----
            estimator_d3 = Estimator(calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3, model_version="v4.0.0")
            calculator_d3 = ASECalculator(estimator_d3)
            atoms.calc = calculator_d3
            
            # ----- CHGNet model -----
            # atoms.calc = CHGNetCalculator(use_device='cpu') 
            
            # ----- Run opt ----------
            cell_filter = ExpCellFilter(atoms)
            opt = BFGS(cell_filter, logfile=None)
            opt.run(fmax=0.02)
            write(f'opt_cifs/{refcode}_opt.cif', atoms) ## Assign folder
            
            # ----- Elastic tensor calc ----------
            diagonal_strains = [-0.015, -0.01, -0.005, 0.005, 0.01, 0.015]
            off_diagonal_strains = [-0.03, -0.02, -0.01, 0.01, 0.02, 0.03]
            elastic_tensor_feature = ElasticTensorFeature(
                diagonal_strains=diagonal_strains, off_diagonal_strains=off_diagonal_strains,
                optimizer=BFGSASEOptFeature(),
                show_logger=True, show_progress_bar=True, tqdm_options={"desc":"elastic tensor"},
                check_symmetry=False
            )
            elastic_tensor = elastic_tensor_feature(atoms)
            elastic_tensor = np.round(elastic_tensor.elastic_tensor, decimals=2)
            C = elastic_tensor
            exp_a = atoms_in.cell.lengths()[0]
            exp_b = atoms_in.cell.lengths()[1]
            exp_c = atoms_in.cell.lengths()[2]
            exp_alpha = atoms_in.cell.angles()[0]
            exp_beta = atoms_in.cell.angles()[1]
            exp_gamma = atoms_in.cell.angles()[2]
            exp_V = atoms_in.cell.volume
            opt_a = atoms.cell.lengths()[0]
            opt_b = atoms.cell.lengths()[1]
            opt_c = atoms.cell.lengths()[2]
            opt_alpha = atoms.cell.angles()[0]
            opt_beta = atoms.cell.angles()[1]
            opt_gamma = atoms.cell.angles()[2]
            opt_V = atoms.cell.volume

            values = [refcode, 
                      C[0,0], C[0,1], C[0,2], C[0,3], C[0,4], C[0,5], 
                      C[1,0], C[1,1], C[1,2], C[1,3], C[1,4], C[1,5],
                      C[2,0], C[2,1], C[2,2], C[2,3], C[2,4], C[2,5],
                      C[3,0], C[3,1], C[3,2], C[3,3], C[3,4], C[3,5],
                      C[4,0], C[4,1], C[4,2], C[4,3], C[4,4], C[4,5],
                      C[5,0], C[5,1], C[5,2], C[5,3], C[5,4], C[5,5],
                      exp_a, exp_b, exp_c, exp_alpha, exp_beta, exp_gamma, exp_V, 
                      opt_a, opt_b, opt_c, opt_alpha, opt_beta, opt_gamma, opt_V]
            
            with open(path_results, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(values)
        except:
            with open(path_error, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([refcode])